![title](img/git.png)

# Я в начале проекта...#

![title](img/before.jpg)

In [4]:
import requests
import math
from functools import wraps
from requests.exceptions import RequestException
from IPython.display import Image

## Декортатор обработки запросов ##


In [2]:
def response_handler(_request):
    """
    Wrapper функция
    Обрабатывает ошибки request запросов
    """

    @wraps(_request)
    def wrapper(self, *args, **kwargs):
        response = None
        try:
            response = _request(self, *args, **kwargs)
            response.raise_for_status()
            return response
        except Exception as e:
            logger_message = (
                f'{e}\n'
                f'URL:{args}\n'
                f'ARGS:{kwargs}\n'
            )
            if isinstance(response, requests.Response):
                logger_message += (
                    f'STATUS_CODE:{response.status_code}\n'
                    f'TEXT:{response.text}'
                )
            logger.error(logger_message)
            raise

    return wrapper


## Пример АПИ класса, по работе со сторонними сервисами ##

In [3]:

class MoeDeloApi:
    """
    Класс для работы с REST API Моё Дело
    https://restapi.moedelo.org/s/?url=/docs#!/
    """

    def __init__(self, api_key):
        self.url = 'https://restapi.moedelo.org/'
        self.session = requests.Session()
        self.session.headers.update({'md-api-key': api_key})
        self.api_key = api_key

    @response_handler
    def _get(self, url, **kwargs):
        return self.session.get(url, **kwargs)

    @response_handler
    def _post(self, url, **kwargs):
        return self.session.post(url, **kwargs)

## Django -fixture ##

Это yaml файлы, в которых можно какие данные дефолтные предусмотреть
Например:
- model: main.client
  pk: 1
  fields:
    username: mts
    password: 123
    mts_token: Token be72b5ad5b7498de457b7bbcc950b3e8c59ee76b
    md_token: 0a3a59bc-c63c-44f1-b281-ef9aa5af61d3
    is_superuser: True
    is_staff: True
    email: novozhilovsv@lad24.ru


## Django-формы ##

In [ ]:
"""
Пример кастомной формы регистрации пользователя
Пример переопределения фалидации поля формы с отправкой запроса к АПИ
"""
from django.contrib.auth.forms import UserCreationForm
from main.models import Client
from django.core.exceptions import ValidationError


class CustomUserCreationForm(UserCreationForm):
    """
    Форма регистрации нового пользователя.
    С обязательными полями: ['username', 'password', 'mts_token', 'md_token']
    """

    class Meta(UserCreationForm.Meta):
        model = Client
        fields = UserCreationForm.Meta.fields + ('mts_token', 'md_token')  # добавляем кастомные поля

    def clean_mts_token(self):
        """
        Кастомный метод валидации вормы регистрации.
        Делается реальный запрос с введенным токеном. И если токен не существует
        в системе МТС, то возникает ошибка валидации
        :return: токен
        """
        mts_token = self.cleaned_data['mts_token']
        if Client.objects.filter(mts_token=mts_token).exists():
            raise ValidationError("МТС токен уже существует")

        mts = MTSApi(api_key=mts_token)

        try:
            mts.tax_get()  # пробуем сделать get-запрос, если нет то вызываем ошибку валидации
        except Exception:
            raise ValidationError('Токен МТС не верный')

        return mts_token

In [ ]:
"""
Пример отрисовфки формы в шаблоне html
"""

In [ ]:
<form class="registration_form_class" action="#" id="registration_form" method="POST">
              {% csrf_token %}
            <p class="form_class_name-field"> Имя</p>
              {{ form.username }}
            <p class="form_class_name-field"> MTS Token</p>
              {{ form.mts_token }}
              {% if form.mts_token.errors %}
                  <p class="registration_form_class_error active-error">
                <font size="2" color="ff0000">{{ form.mts_token.errors.as_text}}</font>
                </p>
              {% endif %}
            <p class="form_class_name-field"> MD Token</p>
              {{ form.md_token }}
                {% if form.md_token.errors %}
                  <p class="registration_form_class_error active-error">
                    <font size="2" color="ff0000">{{ form.md_token.errors.as_text}}</font>
                    </p>
                {% endif %}
            <p class="form_class_name-field"> Пароль</p>
              {{ form.password1 }}
            <p class="form_class_name-field">Подтверждение пароля</p>
              {{ form.password2 }}
</form>

![title](img/form_valid.png)

In [ ]:
"""
Пример переопределения стилей у полей формы
"""
from django import forms
from django_celery_beat.models import PeriodicTask


class SchedulerForm(forms.ModelForm):
    """
    Форма натсроек расписания обмена
    """
    name = forms.CharField(widget=forms.TextInput(attrs={'style': 'display:none'})) # тут я спецом скрыл поле, 
                                                                    #     т.к. оно не редактирумое по задумке

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fields['enabled'].widget.attrs.update({'id': "cbx-1", 'style': "display:none"}) 

    class Meta:
        model = PeriodicTask
        fields = ('name', 'interval', 'enabled')

In [ ]:
"""
Пример орграничение вывода в choise_field (по связанным полям). 
До этого выводились все вобще строки в том числе и по другим пользователям
"""
class MdMtsForm(forms.ModelForm):
    """
    Форма установки связи между кассами МД и МТС
    """

    def __init__(self, *args, **kwargs):
        user = kwargs.pop('user')
        super().__init__(*args, **kwargs)
        if user:
            self.fields['cash_md'].queryset = CashierMd.objects.filter(client=user)
            self.fields['stock_md'].queryset = StockMd.objects.filter(client=user)
            self.fields['trade_mts'].queryset = TradeUnitMts.objects.filter(client=user)

    class Meta:
        model = RelationshipStockCashierTradeUnit
        fields = ['cash_md', 'stock_md', 'trade_mts']

## Django - модели ##

In [ ]:
"""
* установки ограничений для базы уникальности двух полей
* Отображение названия таблицы в Админки
* Опция каскадного удаления
"""


class CashierMd(models.Model):
    """
    Класс описывающий 'операционные кассы' системы Мое Дело.
    """

    id_cash_md = models.IntegerField(unique=True, verbose_name='ID в системе МД')
    created = models.DateTimeField(auto_now_add=True)
    is_main = models.BooleanField()
    name = models.CharField(max_length=128, verbose_name='Название')
    client = models.ForeignKey('main.Client', on_delete=models.CASCADE, verbose_name='Пользователь')

    def __str__(self):
        return self.name

    class Meta:
        verbose_name = 'Операционные кассы МД'
        verbose_name_plural = 'Операционные кассы МД'
        unique_together = ('id_cash_md', 'client')


In [ ]:
"""
Пример создания своего UserManagera c доп. полями
И связывание его с с кастомной моделью пользователя Client
"""
from django.contrib.auth.models import AbstractUser, BaseUserManager
from django.db import models
from django.db.models.signals import post_save


class MyUserManager(BaseUserManager):
    """
    Переопределенный класс, с добавленными обязательными полями токенов.
    """

    def create_user(self, email, username, password, mts_token, md_token):
        if None in (mts_token, md_token, email):
            raise ValueError('Users must have an email address, and tokens')

        user = self.model(
            email=self.normalize_email(email),
            username=username,
            mts_token=mts_token,
            md_token=md_token
        )

        user.set_password(password)
        user.save(using=self._db)
        return user

    def create_superuser(self, email, username, password, mts_token, md_token):
        user = self.create_user(
            email,
            username=username,
            password=password,
            mts_token=mts_token,
            md_token=md_token
        )
        user.is_admin = True
        user.is_superuser = True
        user.is_staff = True
        user.save(using=self._db)
        return user


class Client(AbstractUser):
    """
    Класс описывающий клиентов. К стандартным полям пользователя добавлены
    обязательные md_token, mts_token - токены необходимые для обращения к
    АПИ товароучетных систем МТС и Мое Дело. Эта же модель используется для
    авторизации.
    ВАЖНО!!! При вызове метода save - срабатывает сигнал по синхранизации касс и складов.
    """

    mts_token = models.CharField(unique=True, max_length=128)
    md_token = models.CharField(unique=True, max_length=128)
    objects = MyUserManager()

    USERNAME_FIELD = 'username'
    REQUIRED_FIELDS = ['md_token', 'mts_token', 'email']

# Тут инициализируем сигнал после сохранения пользователя в базу инициализируется задача new_user_start...
from main.signals.new_user_start_sync import new_user_start_sinc
post_save.connect(new_user_start_sinc, sender=Client)

![title](img/signal_tree.png)

### Сигналы ###

In [ ]:
"""
Сама функция сигнала
"""
def new_user_start_sinc(sender, **kwargs):
    """
    По сигналу о создании нового пользователя
    Стартуют задачи по синхронизации
    """
    instance = kwargs.get('instance')
    sync_all.delay(client_id=instance.pk)  # тут мы запускаем асинхронную задачку
    sync_data_between_mts_md.delay(client_id=instance.pk)

In [ ]:
"""
Пример переопределения методов save и delete модели
"""
class RelationshipStockCashierTradeUnit(models.Model):
    """
    Связующий класс-таблица, описывающий связку Подразделения МТС с
    сущносятми операционная касса и склад из системы Мое Дело.
    Задано ограничение, предполагащие одну запись в таблице для
    одного подразделения МТС.
    """

    client = models.ForeignKey('main.Client', verbose_name='Клиент', on_delete=models.CASCADE)
    cash_md = models.ForeignKey('main.CashierMd', verbose_name='Опер.касса МД', on_delete=models.DO_NOTHING)
    stock_md = models.ForeignKey('main.StockMd', verbose_name='Склад МД', on_delete=models.DO_NOTHING)
    trade_mts = models.ForeignKey('main.TradeUnitMts', verbose_name='Подразделение МТС', on_delete=models.DO_NOTHING)
    created = models.DateTimeField(auto_now_add=True)

    class Meta:
        verbose_name = 'Таблица связей объектов'
        verbose_name_plural = 'Таблица связей объектов'
        unique_together = ('client', 'trade_mts')

    def _create_relay_task(self, created=False):
        """
        После создания новой связи между объектами
        МТС и МОЕ ДЕЛО, создаются две задачки по расписанию
        И Записи в связанных с ними таблицами
        """
        if created:

            schedule, is_created = IntervalSchedule.objects.get_or_create(
                period=CONFIG['DEFAULT_SHEDULER']['period'],
                every=CONFIG['DEFAULT_SHEDULER']['every'])
            task_upload_items, is_created = PeriodicTask.objects.get_or_create(
                interval=schedule,
                name=f'Выгрузка товаров из Мое Дело в МТС_{self.client_id}',
                task='main.tasks.load_items.load_items',
                args=json.dumps([self.client_id]))
            RelationClientPeriodicTask.objects.get_or_create(
                client_id=self.client_id, periodic_task_id=task_upload_items.id)
            task_upload_sales, is_created = PeriodicTask.objects.get_or_create(
                interval=schedule,
                name=f'Загрузка продаж из МТС в Мое Дело_{self.client_id}',
                task='main.tasks.load_sales.load_sales',
                args=json.dumps([self.client_id]))
            RelationClientPeriodicTask.objects.get_or_create(
                client_id=self.client_id,
                periodic_task_id=task_upload_sales.id)

    def _delete_relayted_task(self):
        count = RelationshipStockCashierTradeUnit.objects.filter(
            client=self.client).count()

        if count == 0:
            tasks = RelationClientPeriodicTask.objects.filter(
                client=self.client)
            tasks_id = [
                i.periodic_task.id for i in tasks if i.periodic_task.name != f'Синхронизация касс и складов_{self.client.id}'
            ]
            PeriodicTask.objects.filter(id__in=tasks_id).delete()

    def save(self, *args, **kwargs):
        created = self.pk is None
        try:
            super(RelationshipStockCashierTradeUnit, self).save(*args, **kwargs)
            self._create_relay_task(created)
        except IntegrityError as e:
            logger.error(e)

    def delete(self, *args, **kwargs):
        """
        При удаление последней записи в таблице, удаляются все связанные записи
        В том числе и в таблицах связанные с задачами по синхронизации.
        """
        super(RelationshipStockCashierTradeUnit, self).delete(*args, **kwargs)
        self._delete_relayted_task()

## CELERY - TASK ##

In [ ]:
"""
Пример создания задачки
"""

![title](img/task_tree.png)

In [ ]:
"""
Важно таски должны быть в __init__.py иначе их не найдет celery

from .cashiers_md import *
from .stoks_md import *
from .trade_unit import *
from .load_sales import *
from .load_items import *
"""

In [ ]:
"""
Пример таски которая, возвращает chord из из вдух груповых тасок
Задачка собираемая в групу должна быть обернута сигнатурой:
task.s - так она не будет вызываться, и такую задачу можно просто добавить в список задач.
task.delay - вызовет задачу в асинхронном режиме
"""

@app.task(trail=True) # так объявляется таска
def handle_session(parrent_id, session_id, client_id):
    """
    Задачка по обработки данных сессии полученной от МТС
    :param parrent_id - :
    :param session_id - :
    Формирует готовые отчеты, которые передает другой задаче не отправку
    """
    last_session = SessionModel.objects.filter(parent_task__client_id=client_id)
    list_ids = [i.session_id for i in last_session]
    if str(session_id) in list_ids:
        return f'Сессия № {session_id} обрабатывалась ранее'

    parent_task = SaleTaskModel.objects.get(task_id=parrent_id)
    new_session = SessionModel.objects.create(session_id=session_id, parent_task=parent_task)

    z_report = new_session.data['data'][0].get('z_report')
    documents = new_session.data['data'][0].get('documents')

    trade_mts_obj = TradeUnitMts.objects.get(client=parent_task.client, id_mts=z_report.get('shop_id'))
    relation_obj = RelationshipStockCashierTradeUnit.objects.get(client=parent_task.client, trade_mts=trade_mts_obj)

    report_jobs = new_session.create_report_jobs(
        z_report, documents, md_token=relation_obj.client.md_token, cash_id=relation_obj.cash_md.id_cash_md)

    all_items = new_session.create_all_items_job(
        documents, mts_token=relation_obj.client.mts_token, client_id=relation_obj.client.id)

    return chord([group(report_jobs), group(all_items)],  # Можно списком собрать пачку груповых задачек
                 send_retail_report.s(z_report=z_report, # В эту задачку передаем результат
                                      md_token=trade_mts_obj.client.md_token,
                                      stock_id=relation_obj.stock_md.id_stock_md))()

In [ ]:
"""
Пример таски которая принимает результаты двух груповых тасок
"""
@app.task(
    default_retry_delay=10,
    max_retries=5,
    autoretry_for=(RequestException,),
    retry_backoff=True
)
def send_retail_report(*args, **kwargs):  # В args как раз прилетают результаты от задачек, 
                                          # в kwargs можно свои передать параметры
    """
    Задача по отправке отчета о продажах
    :param args: - результаты предыдущих задач (списко товаров, и кассовых операций)
    :param kwargs:  z-отчет , md_token, store_list
    :return: результат запроса к АПИ Мое Дело
    """
    revenue_report = []
    items = []

    for spam in args[0]:
        if isinstance(spam, (tuple, list)):
            revenue_report.append({'CashierId': spam[1], 'RetailRevenueId': f'{spam[0]}'})
        elif isinstance(spam, dict):
            if spam['Name'] != 'Свободная продажа':
                items.append(spam)

    md = MoeDeloApi(api_key=kwargs['md_token'])
    result = md.send_retail_report(revenue_report, items, **kwargs)
    return result

In [ ]:
"""
Собсвтенные фильтры в шаблонах (templatetags)
"""

In [ ]:
from django.template.defaulttags import register


@register.filter(name='get_item')
def get_item(dictionary, key):
    """
    Кастомный фильтр для получение значение по ключу в шаблоне
    :param dictionary: словарь в шаблоне
    :param key: искомый ключ
    :return: значение
    """
    return dictionary.get(key)

@register.filter(name='split')
def split(value, arg):
    """
    Кастомный фильтр удаление окончания '_4' при ренедринге в шаблоне.
    Окончание означает id пользователя. И наличие этого окончания обеспечивет уникальность имени задачки
    :param value: строка названия
    :param arg: символ по которому разбиваем строку
    :return: чистое название
    """

    return value.split(arg)[0]

In [ ]:
"""
В шаблоне это выглядит так
"""
<div class="table-data-two_table-cell">{{ translate_interval|get_item:row.periodic_task.interval.period }}</div>
<div class="table-data-two_table-cell-1">{{ row.periodic_task.name|split:"_" }}</div>

## Pytest - django and TDD ##

![title](img/goat.png)

![title](img/test_goat.png)

### Суть подхода TDD ###

![title](img/tdd_step1.png)

![title](img/tdd_step2.png)

In [ ]:
"""
Pytest - fixture
Фикстуры создаются в файлике c
"""


![title](img/tdd_cycle.png)

In [ ]:
"""
Пример отправки подтверждения на почту во вьюхе
"""

![title](img/send_mail_view.png)

Пример генерации уникального ID, который шлется потом на email для подтверждения

![title](img/uuid.png)

LOGGING

![title](img/logging.png)

Функциональный тест отправки уведомления на email!

![title](img/check_mail.png)

### MOCK ###

![mock](img/mock.png)

### Mock class-level ###

![mock_class](img/mock_class.png)

### email - config ###

![email](img/email.png)

### logout ###

![logout](img/logout.png)

### Тестирование создание сессии для пользователя ###

![session](img/session.png)

### Я в конце проекта.. ###

![title](img/after.jpg)

type